In [11]:
import pandas as pd

import psycopg2
from psycopg2 import errorcodes, OperationalError

In [12]:
df_aemet = pd.read_csv("datos/df_aemet_merged.csv", index_col=0)
df_places = pd.read_csv("datos/df_places_indexed.csv", index_col=0)
df_municipios = pd.read_csv("datos/df_municipios.csv", index_col=0)

In [13]:
df_places.drop(df_places[df_places["fsq_id"].duplicated()].index, inplace = True)

In [14]:
def conectar():
    try:
        connection = psycopg2.connect(
            database = "AEMET-lab",
            user = "postgres",
            password = "admin",
            host = "localhost",
            port = "5432"
        )
    except OperationalError as e:
        if e.pgcode == errorcodes.INVALID_PASSWORD:
            print("La constraseña es errónea.")
        elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
            print("Error de conexión")
        else:
            print(f'Error:{e}')
    return connection

def query_fetch(connection, query_text):
    cursor = connection.cursor()
    cursor.execute(query_text)
    result = cursor.fetchall()
    cursor.close()
    connection.close()
    return result

def query_commit(connection, query_text, *valores):
    cursor = connection.cursor()
    cursor.execute(query_text, *valores)
    connection.commit()
    cursor.close()
    connection.close()
    return print("Done!")

def query_commit_many(connection, query_text, *valores):
    cursor = connection.cursor()
    cursor.executemany(query_text, *valores)
    connection.commit()
    cursor.close()
    connection.close()
    return print("Done!")

In [15]:
conexion = conectar()

query_tables = '''
CREATE TABLE IF NOT EXISTS municipios(
id_municipio VARCHAR(100) primary key, 
latitude DECIMAL(10,5),
longitude DECIMAL(10,5), 
nombre VARCHAR(100) unique not null
);

CREATE TABLE IF NOT EXISTS daystages(
id_daystage VARCHAR(100) primary key, 
descripcion TEXT
);

CREATE TABLE IF NOT EXISTS lugares(
id_lugar VARCHAR(100) primary key, 
latitude DECIMAL(10,5),
longitude DECIMAL(10,5), 
nombre VARCHAR(100) not null,
categoria VARCHAR(100),
closed_bucket VARCHAR(100),
distancia DECIMAL(10,2),
direccion VARCHAR(300),
link VARCHAR(300),
id_municipio VARCHAR(100) not null,
foreign key (id_municipio)
    references municipios(id_municipio)
);

CREATE TABLE IF NOT EXISTS tiempo(
id_tiempo serial primary key,
fecha DATE,
cielo VARCHAR(200),
temperatura DECIMAL(5,2),
sens_térmica DECIMAL(5,2),
humedad_rel DECIMAL(10,2),
precipitacion DECIMAL(10,2),
prob_precip VARCHAR(200),
nieve DECIMAL(10,2),
prob_nieve VARCHAR(100),
prob_tormenta VARCHAR(200),
vel_viento DECIMAL(10,2),
racha_max DECIMAL(10,2),
dir_viento VARCHAR(100),
id_daystage VARCHAR(100),
id_municipio VARCHAR(100),
foreign key (ID_daystage)
    references daystages(id_daystage),
foreign key (id_municipio)
    references municipios(id_municipio)
);
'''

query_commit(conexion, query_tables)

Done!


In [16]:
ds_ids = ["madrugada", "mediodia", "tarde", "noche"]
ds_desc = ['''
            Momento de quietud y calma previa al amanecer, ideal para capturar escenas 
           nocturnas profundas o el inicio de un nuevo día con los primeros rayos del sol. 
           La luz tenue y la atmósfera tranquila crean una estética única.
           ''',
           '''
            La luz es intensa y directa, lo que genera sombras marcadas. 
            Perfecto para exteriores vibrantes y escenas dinámicas bajo el sol pleno, 
            aunque puede requerir control de iluminación para evitar reflejos no deseados.
            ''',
            '''
            Luz suave y cálida que comienza a caer, lo que permite transiciones dramáticas en las escenas. 
            Ideal para capturar la "hora dorada", donde los tonos anaranjados y rosados dominan, 
            agregando calidez y profundidad a las tomas.
            ''',
            '''
            Ambiente oscuro, dominado por luces artificiales o naturales como la luna. 
            Propicio para escenas misteriosas o íntimas, 
            donde la iluminación es clave para generar efectos atmosféricos o dramáticos.
            '''      
           ]

ds_vals = []
for id, desc in zip(ds_ids, ds_desc):
    ds_vals.append((id, desc))

In [17]:
query_values_daystages = '''insert into daystages(id_daystage, descripcion)
                            values (%s,%s)'''

conexion = conectar()
query_commit_many(conexion, query_values_daystages, ds_vals)

Done!


In [18]:
df_municipios.head(1)

,municipio_id,latitude,longitude,nombre
0,acebeda-la,41.086958,-3.624399,La Acebeda


In [19]:
query_values_municipios = '''insert into municipios(id_municipio, latitude, longitude, nombre)
                                values (%s,%s,%s,%s)'''

m_ids = df_municipios["municipio_id"].values
m_lats = df_municipios["latitude"].values
m_long = df_municipios["longitude"].values
m_name = df_municipios["nombre"].values

m_values = []
for i,la,lo,na in zip(m_ids, m_lats, m_long, m_name):
    m_values.append((i,la,lo,na))

conexion = conectar()
query_commit_many(conexion, query_values_municipios, m_values)

Done!


In [20]:
lista_tuplas = []
for val in df_places.values:
    lista_tuplas.append(tuple(val))

print(lista_tuplas[0])

('ajalvir', 'Park', '4d3c6c5b84d46ea87dddfd5c', 'LikelyOpen', 563.0, '/v3/places/4d3c6c5b84d46ea87dddfd5c', 'Diverjungla', 'Calle Segovia, 20 (Pol. Ind. Donada), 28864 Ajalvir Comunidad de Madrid', 40.529921, -3.484341)


In [21]:
query_values_lugares = '''insert into lugares(id_municipio, categoria, id_lugar, closed_bucket, distancia, link, nombre, direccion, latitude, longitude)
                                values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''

conexion = conectar()
query_commit_many(conexion, query_values_lugares, lista_tuplas)

Done!


In [22]:
conexion = conectar()
print(query_fetch(conexion, '''
                        SELECT column_name
                        FROM INFORMATION_SCHEMA.COLUMNS
                        WHERE TABLE_NAME = 'tiempo'
                    '''))

[('id_tiempo',), ('fecha',), ('cielo',), ('temperatura',), ('sens_térmica',), ('humedad_rel',), ('precipitacion',), ('prob_precip',), ('nieve',), ('prob_nieve',), ('prob_tormenta',), ('vel_viento',), ('racha_max',), ('dir_viento',), ('id_daystage',), ('id_municipio',)]


In [23]:
lista_tuplas = []
for val in df_aemet.values:
    lista_tuplas.append(tuple(val))
print(lista_tuplas[0])

('acebeda-la', '2024-08-29', 'mediodia', 'Nubes altas', 'Muy alta', 'Baja', 'Alta', 'S', 21.0, 21.0, 25.0, 0.0, 0.0, 69.0, 7.0)


In [24]:
query_values_tiempo = '''insert into tiempo(id_municipio, fecha, id_daystage, cielo, prob_precip, prob_nieve, prob_tormenta, dir_viento, temperatura, sens_térmica, racha_max, precipitacion, nieve, humedad_rel, vel_viento )
                                values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''

conexion = conectar()
query_commit_many(conexion,query_values_tiempo, lista_tuplas)

Done!


### 1.¿Cuáles son los lugares con la menor probabilidad de tormenta durante la tarde del 30 de agosto de 2024?

Para asegurar rodajes al aire libre sin riesgos climáticos en esa fecha específica.

In [25]:
query = '''
        select m.nombre from municipios m 
        where m.id_municipio in (select id_municipio from tiempo t
							where t.fecha = '2024-08-30' 
							and t.id_daystage = 'tarde' 
							and t.prob_tormenta in ('Baja'));
        '''

conexion = conectar()
query_fetch(conexion, query)

[]

No hay ningún municipio con prob tormenta baja, probemos con media.

In [26]:
query = '''
        select m.nombre from municipios m 
        where m.id_municipio in (select id_municipio from tiempo t
							where t.fecha = '2024-08-30' 
							and t.id_daystage = 'tarde' 
							and t.prob_tormenta in ('Media'));
        '''

conexion = conectar()
df_1 = pd.DataFrame(query_fetch(conexion, query))
print(df_1.shape[0])
df_1.sample(3)

54


,0
32,Brunete
10,Daganzo de Arriba
12,San Sebastián de los Reyes


Hay 54 municipios que tenían probabilidad baja.

También podemos hacerlo sin anidar:

In [27]:
query = '''
        select m.nombre, t.prob_tormenta from municipios m 
        inner join tiempo t on t.id_municipio = m.id_municipio 
        where t.id_daystage = 'tarde' and t.fecha = '2024-08-30' and t.prob_tormenta = 'Media';
        '''
conexion = conectar()
df_11 = pd.DataFrame(query_fetch(conexion, query))

df_11.head()

,0,1
0,Ajalvir,Media
1,El Álamo,Media
2,Alcobendas,Media
3,Aldea del Fresno,Media
4,Batres,Media


### 2.¿Qué municipios tienen temperaturas por debajo de la mediana de temperatura de todos los municipios en la noche del 29 de agosto de 2024?
Para identificar ubicaciones con climas fríos en la noche de esa fecha.

In [28]:
query = '''
select m.nombre from municipios m 
where m.id_municipio in (select id_municipio from tiempo t 
							where t.fecha = '2024-08-29'
							and t.id_daystage = 'noche'
							and t.temperatura < (select percentile_cont(0.5) 
												within group(order by temperatura) 
												from tiempo));
                                                '''
conexion = conectar()
df_2 = pd.DataFrame(query_fetch(conexion, query))

df_2.sample(5)

,0
13,Chinchón
26,Moraleja de Enmedio
79,Boadilla del Monte
77,Navacerrada
53,Robledo de Chavela


### 3.¿En qué municipios la velocidad del viento ha sido superior a la velocidad promedio de todos los municipios durante el día 31 de agosto de 2024?

Para evitar filmaciones en lugares donde los fuertes vientos pueden ser un problema.

In [29]:
query = '''select m.id_municipio, t.vel_viento from municipios m
inner join tiempo t on m.id_municipio = t.id_municipio
where t.vel_viento > (select AVG(vel_viento) from tiempo 
                       where fecha = '2024-08-31');'''

conexion = conectar()
df_3 = pd.DataFrame(query_fetch(conexion, query))

df_3.sample(5)

,0,1
779,torrelaguna,12.33
294,daganzo-de-arriba,12.67
1,ajalvir,9.00
212,chinchon,16.60
457,moraleja-de-enmedio,20.57


### 4.¿Cuáles son los lugares a menos distancia de la media a Valdemoro donde la velocidad del viento fue menor que el promedio de todos los lugares el 31 de agosto de 2024?


In [30]:
query = '''
select nombre, distancia from lugares l 
where id_municipio in (select id_municipio from tiempo t
						where vel_viento < (select avg(vel_viento) from tiempo))
and id_municipio = (select id_municipio from municipios m
						where m.nombre = 'Valdemoro')
and l.distancia < (select avg(distancia) from lugares l
						where l.id_municipio=(select id_municipio from municipios m
						where m.nombre = 'Valdemoro'))
'''

conexion = conectar()
df_4 = pd.DataFrame(query_fetch(conexion, query))

df_4.sample(5)

,0,1
1,Parque Forestal Cerro del Castillo,580.00
10,Parque de las Eras,1027.00
7,Parque Infantil Hojarasca,870.00
6,Parque Duque de Ahumada,794.00
3,Parque de Extremadura,628.00


### 5.¿Cuáles son los parques a menos de 2 km de cada localización que tienen una probabilidad de precipitación baja para el mediodia el día 31 de agosto de 2024?

Para encontrar locaciones al aire libre con buen clima en el verano.

In [31]:
query = '''select l.nombre, m.nombre from lugares l
join municipios m on m.id_municipio = l.id_municipio 
where l.id_municipio in (select t.id_municipio from tiempo t
						where t.prob_precip = 'Baja'
						and t.fecha = '2024-08-31'
						and t.id_daystage = 'mediodia')
and l.distancia < 2000;
					'''

conexion = conectar()
df_5 = pd.DataFrame(query_fetch(conexion, query))

df_5.sample(5)

,0,1
806,Parque Antonio Machado,Villanueva del Pardillo
119,Parque Infantil,Alcorcón
637,Pinar de las Rozas,Las Rozas de Madrid
199,Parque de las Avenidas,Buitrago del Lozoya
425,Estatua de Felipe IV,Madrid
